# 01 Importação dos pacotes e bibliotecas

In [ ]:
!pip install --upgrade google-api-python-client
!pip install langdetect
!pip install unidecode
!pip install langdetect

In [ ]:
import pandas as pd
import numpy as np
import unidecode
import itertools
import nltk
import time
import re

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from googleapiclient.discovery import build
from six.moves import urllib
from datetime import datetime
from nltk.stem import *

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

# 02 Autenticação no ambiente Google (BiQuery / Drive)

In [ ]:
# autenticação BQ
from google.colab import auth

auth.authenticate_user()

In [ ]:
# autenticação drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 03 Consultas Youtube API

## 03.a Cria dicionário da lingua portuguesa

In [ ]:

# Cria lista para armazenar palavras do dicionário
dicionario = []

# Cria objeto para stemming das palavras
stemmer = nltk.stem.RSLPStemmer()

# Importar das URLs abaixo dicionários com palavras da língua portuguesa

# 1º Dicionário
url = "https://www.ime.usp.br/~pf/dicios/br-utf8.txt"
file = urllib.request.urlopen(url)

for line in file:
  decoded_line = line.decode("utf-8")
  dicionario.append(decoded_line.rstrip("\n"))

# 2º Dicionário
url = "http://200.17.137.109:8081/novobsi/Members/cicerog/disciplinas/introducao-a-programacao/arquivos-2016-1/algoritmos/Lista-de-Palavras.txt"
file = urllib.request.urlopen(url)

for line in file:
  decoded_line = line.decode("utf-8")
  dicionario.append(decoded_line.rstrip("\n"))

# Transforma o texto para caixa baixa
dicionario = [" ".join(x.lower() for x in x.split()) for x in dicionario]

# Remoção de pontuação
dicionario = [x.replace('[^\w\s]','') for x in dicionario]
dicionario = [x.replace('-','') for x in dicionario]

# Remoção dos acentos
dicionario = [unidecode.unidecode(x) for x in dicionario]

# Stemm
dicionario = [stemmer.stem(x) for x in dicionario]

# Remove duplicatas
dicionario = list(set(dicionario))

## 03.b Consulta vídeos

In [ ]:

# declaração da chave API
api_key = 'Insira sua chave API aqui'

# token que será procurado
query = 'CryptoCars'

# ticker do token que será procurado
nomeCrypto = 'CCAR'

# criando objeto para realizar consultas no Youtube
youtube_object = build('youtube', 'v3', developerKey = api_key)

# chamando o método search.list para recuperar resultados de pesquisa do youtube
search_keyword = youtube_object.search().list(q=query, 
                                              part='id, snippet',
                                              regionCode='BR', # consultas na região do Brasil
                                              relevanceLanguage='pt', # idioma em português
                                              maxResults=50).execute()

# extrai os resultados
results = search_keyword.get('items', [])

Como o comentário pode vir desconfigurado, são necessários alguns ajustes antes da ingestão dos dados.

In [ ]:

# Importação das stopwords
stop = stopwords.words('portuguese')
stop.extend(['nao','boa','top','peter'])

def transform_text(sentence):
  '''
  Input:
      sentence: frases que serão pré-configuradas para um padrão mais legível
  Output:
      sentence: frases já pré-configuradas
  '''
  # Transforma o texto para caixa baixa 
  sentence = sentence.lower()

  # Remove & por conta da falha em algumas decodificações
  sentence = ' '.join(map(str,[x for x in sentence.split() if x[0] != '&']))

  # Remoção de pontuação
  sentence = sentence.replace('[^\w\s]',' ')
  sentence = sentence.replace('\r',' ')
  sentence = sentence.replace('|',' ')

  # Remoção dos acentos
  sentence = unidecode.unidecode(sentence)

  # Remoção de excesso de espaços em branco
  sentence = re.sub(' +', ' ', sentence)

  return sentence

In [ ]:

def youtube_search_keyword(results):
  '''
  Input:
      results: resultado da chamada API
  Output:
      df: conjunto de dados com informações sobre os vídeos consultados
  '''
  # lista vazia para armazenar:
  # título do vídeo, id do vídeo, nome do canal, data da publicação do vídeo
  title = []
  videoId = []
  channelTitle = []
  description = []
  publishedAt = []
  
  # extração de informações necessárias de cada objeto do resultado da chamada API
  for result in results:
    # objeto de resultado de vídeo
    if result['id']['kind'] == 'youtube#video':
      title.append(transform_text(result['snippet']['title']))
      videoId.append(result['id']['videoId'])
      channelTitle.append(transform_text(result['snippet']['channelTitle']))
      description.append(transform_text(result['snippet']['description']))
      publishedAt.append(datetime.fromisoformat(result['snippet']['publishedAt'][:-1]).strftime('%Y-%m-%d'))

  df = pd.DataFrame(
    {'title': title,
     'videoId': videoId,
     'channelTitle': channelTitle,
     'description': description,
     'publishedAt': publishedAt
    })

  return df

# cria conjunto de dados com videos
df_videos = youtube_search_keyword(results)

## 03.c Consulta comentários do vídeos

In [ ]:

def remove_repeated_letters(sentence):
  '''
  Input:
      sentence: frase com palavras que podem ter letras repetidas indevidamente
  Output:
      newSentence: frase configurada (sem repetição indevida de letras)
  '''
  newSentence = []

  for word in sentence.split():

    if stemmer.stem(word) not in dicionario:
      word = ''.join(c[0] for c in itertools.groupby(word))
      newSentence.append(word)
    else:
      newSentence.append(word)

  newSentence = " ".join(newSentence)

  return newSentence

In [ ]:

# Lista para NÃO submeter a Stemm
notStemm = ['entr','compr','scam','scan']

def transform_text_toStemmed(sentence):
  '''
  Input:
      sentence: frase com texto original (pré-configurado)
  Output:
      textStemmed: frase preparada para PLN
  '''
  textStemmed = sentence

  nmeTOKEN = 'crypto' + nomeCrypto

  # Ajuste para preservar o nome do TOKEN do jogo
  textStemmed = textStemmed.replace(nomeCrypto.lower(),nmeTOKEN)

  # Remove hastags (alguns youtubers lançam sorteios com hashtag que comprometem a analise [sorteiobold ] )
  textStemmed = ' '.join(map(str,[x for x in textStemmed.split() if x[0] != '#']))
  
  # Remove & por conta da falha em algumas decodificações
  textStemmed = ' '.join(map(str,[x for x in textStemmed.split() if x[0] != '&']))

  # Remoção de pontuação
  textStemmed = textStemmed.replace('!',' ')
  textStemmed = textStemmed.replace('.',' ')
  textStemmed = textStemmed.replace(',',' ')
  textStemmed = textStemmed.replace('=',' ')
  textStemmed = textStemmed.replace('$',' ')
  textStemmed = textStemmed.replace(')',' ')
  textStemmed = textStemmed.replace('(',' ')
  textStemmed = textStemmed.replace('#',' ')
  textStemmed = textStemmed.replace(':',' ')
  textStemmed = textStemmed.replace(';',' ')
  textStemmed = textStemmed.replace('&quot',' ')
  textStemmed = textStemmed.replace('&#39',' ')

  # Remoção das stop words
  stopwords = [unidecode.unidecode(x) for x in stop]

  textStemmed = ' '.join(map(str,[x for x in textStemmed.split() if x not in stopwords]))

  # Stemm
  textStemmed = ' '.join(map(str,[stemmer.stem(x) if stemmer.stem(x) not in notStemm else x for x in textStemmed.split()]))

  # Remoção de letras repetidas (uma após a outra), se nao estiver no dicionario
  textStemmed = ' '.join(map(str,[remove_repeated_letters(x) for x in textStemmed.split()]))

  # Remoção de palavras irrelevantes (dois caracteres ou menos)
  textStemmed = ' '.join(map(str,[x for x in textStemmed.split() if len(x)>2]))

  # Remoção de excesso de espaços em branco
  textStemmed = ' '.join(map(str,[re.sub(' +', ' ', x) for x in textStemmed.split()]))

  # TOKEN modificado
  nmeTOKENadj = ''.join(sorted(set(nmeTOKEN), key=nmeTOKEN.index))
  nmeTOKENadj = stemmer.stem(nmeTOKENadj)

  # Recuperando alterações feitas no ajuste de nome do TOKEN
  textStemmed = textStemmed.replace(nmeTOKENadj,nmeTOKEN)

  return textStemmed

In [ ]:
def remove_from_string(textStemmed,textDisplay):
  '''
  Input:
      textStemmed: frase preparada para PLN
      textDisplay: frase com texto original (pré-configurado)
  Output:
      checkFlg: se maior que 0, o comentario nao sera considerado por
                nao passar nos filtros de relevancia
  '''
  
  checkFlg = 0

  # Remove comentários sem texto
  if textStemmed ==  '':
    checkFlg += 1

  # Remove comentários extensos
  if len(textStemmed) > 100:
    checkFlg += 1

  # Remove comentários de uma palavra
  if len(textStemmed.split()) == 1:
    checkFlg += 1

  # Remove perguntas (nao sao relevantes)
  if '?' in textDisplay:
    checkFlg += 1

  # Remove comentarios com finalidade que nao auxilia na analise de sentimento
  # Ex: 'Otimo video','Excelente conteudo'.
  remove = ['vide','conteud','parab','fal sobr','analis','explic']

  for element in remove:
    if element in textStemmed:
      checkFlg += 1

  return checkFlg

In [ ]:
# {execuçao de aprox. 04-05 min}

def video_comments(list_title, list_video_id, list_authorDisplayName):
  '''
  Input:
      list_title: lista com títulos dos vídeos a serem consultados
      list_video_id: lista com id dos vídeos a serem consultados
      list_authorDisplayName: lista com nome dos canais
  Output:
      df_comments: conjunto de dados com informações sobre os comentários dos vídeos consultados
  '''
  df_comments = pd.DataFrame()

  for nmetitle, video_id, authorDisplayName in zip(list_title, list_video_id, list_authorDisplayName):

    count_item = 0

    # requisição para API do youtube para determinado id de video
    request=youtube_object.commentThreads().list(part='snippet,replies',
                                                 order= 'relevance',
                                                 videoId=video_id
                                                 )

    title = []
    channelTitle = []
    publishedAt = []
    textDisplay = []
    textStemmed = []
    likeCount = []
    totalReplyCount = []

    # iterar requisição para API
    while request:

      # como alguns vídeos possuem mais comentários que outros,
      # para evitar uma concentração de comportamento/percepção muito grande
      # em um mesmo vídeo, cria-se um limite de 500 comentários por video
      if count_item > 500:
        break

      # se não houver problema na execução da requisição:
      try:
        
        video_response = request.execute()
      
        for item in video_response['items']:

          # o comentário não pode ser do dono do canal (qse sempre não é relevante)
          if item['snippet']['topLevelComment']['snippet']['authorDisplayName'] != authorDisplayName:

            # data da publicação do comentário
            datePub = item['snippet']['topLevelComment']['snippet']['publishedAt']
            datePub = datetime.fromisoformat(datePub[:-1]).strftime('%Y-%m-%d')

            # comentário (submetido a função transform_text() )
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comment = transform_text(comment)
            comment = re.sub(re.compile('<.*?>'), '', comment)

            # comentário (submetido a função transform_text_toStemmed() )
            commentStemmed = transform_text_toStemmed(comment)

            # número de curtidas para o comentário
            likes = item['snippet']['topLevelComment']['snippet']['likeCount']
            likes = int(likes)

            # número de réplicas para o comentário
            replycount = item['snippet']['totalReplyCount']
            replycount = int(replycount)

            # Motor de regra para validar se o comentário é relevante ou nao
            if remove_from_string(commentStemmed,comment) == 0:

              title.append(nmetitle)
              channelTitle.append(authorDisplayName)
              publishedAt.append(datePub)
              textDisplay.append(comment)
              textStemmed.append(commentStemmed)
              likeCount.append(likes)
              totalReplyCount.append(replycount)
            
              count_item += 1

        # Repete a consulta (se existirem, carrega novos comentários)
        if 'nextPageToken' in video_response:
          request = youtube_object.commentThreads().list_next(request,
                                                              video_response
                                                              )
        else:
          break

      # se a execução der erro (por exemplo: nao há comentarios para o video)
      except:
        # a execução é interrompida e o proximo dado processado
        break

    df = pd.DataFrame(
      {'title': title,
      'channelTitle': channelTitle,
      'publishedAt': publishedAt,
      'textDisplay': textDisplay,
      'textStemmed': textStemmed,
      'likeCount': likeCount,
      'totalReplyCount': totalReplyCount
      })
    
    df.sort_values(by=['likeCount'], ascending=False, inplace=True)

    df_comments = df_comments.append(df)

  df_comments.drop_duplicates(inplace=True)

  return df_comments

# lista com vídeos e nomes dos canais
list_title = list(df_videos['title'])
list_video_id = list(df_videos['videoId'])
list_authorDisplayName = list(df_videos['channelTitle'])

# cria conjunto de dados com comentários dos videos
df_comments = video_comments(list_title, list_video_id, list_authorDisplayName)

In [ ]:
# consumo ao fim do projeto de 370 queries

# 04 Exportação do conjunto de dados para o GCP

In [ ]:
import pandas_gbq

from google.oauth2 import service_account

def save_gbq(df, channel_name, nome_tabela, path_json_key):
  '''
  Input:
      df: conjunto de dados a ser carregado no GCP
      channel_name: nome do canal
      nome_tabela: nome que será dado ao conjunto de dados carregado
      path_json_key: chave json para a autenticaçao do acesso
  '''
  credentials = service_account.Credentials.from_service_account_file(path_json_key)
  pandas_gbq.to_gbq(dataframe = df,
                    destination_table = f'{channel_name}.{nome_tabela}',
                    project_id = 'Insira o nome do seu projeto aqui',
                    credentials = credentials,
                    if_exists='replace')

save_gbq(df_comments, 'youtube', 'df_comments', 'Insira o caminho de seu arquivo json aqui')

1it [00:05,  5.15s/it]
